In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import numpy as np

In [3]:
os.getcwd()

'/Users/regan/Documents/Springboard/Capstone2.0'

In [4]:
#load 2019 play-by-play dataset
pbp2019 = pd.read_csv('2019_CFB_Data.csv', encoding='latin-1')
pbp2019.head()

,gameid,provider,o/u,hometeam,home money,home spread,awayteam,away money,away spread,playid,...,fumble,td,xp result,two pt result,playstring,play str len,complex,max len,max spaces,week
0,401112431,teamrankings,55.0,Florida State Seminoles,-234.0,-6.0,Boise State Broncos,200.0,6.0,102925701,...,True,False,NaN,NaN,James Blackman sacked by Kekaula Kaniho and TE...,223,5,33,5,1
1,401110725,teamrankings,51.0,LSU Tigers,-3019.0,-27.0,Georgia Southern Eagles,1487.0,27.0,101998701,...,True,False,NaN,NaN,Shai Werts sacked by Rashard Lawrence for a lo...,212,5,33,5,1
2,401110723,teamrankings,46.0,Florida Gators,-291.0,-7.0,Miami Hurricanes,244.0,7.0,104966901,...,True,False,NaN,NaN,Jarren Williams sacked by Trey Dean III for a ...,192,5,33,5,1
3,401121935,consensus,49.0,Troy Trojans,NaN,-35.0,Campbell Fighting Camels,NaN,35.0,103979801,...,True,False,NaN,NaN,Team run for a loss of 26 yards for a SAFETY T...,101,5,33,5,1
4,401114228,teamrankings,65.5,Washington State Cougars,-5595.0,-33.0,New Mexico State Aggies,2219.0,33.0,101937404,...,False,False,NaN,NaN,Anthony Gordon pass complete to Brandon Arcona...,63,2,33,5,1


In [11]:
#remove unneeded columns from PBP dataset
pbp2019.columns

Index(['gameid', 'provider', 'o/u', 'hometeam', 'home money', 'home spread',
       'awayteam', 'away money', 'away spread', 'playid', 'activeid',
       'activeteam', 'nonactive team', 'drive #', 'playtype', 'quarter', 'sec',
       'homescore', 'awayscore', 'down', 'distance', 'yardstoendzone', 'zone',
       'qb', 'pass yds', 'receiver', 'recyds', 'runner', 'runyds', 'kicker',
       'kickyds', 'kickreturner', 'kickretyds', 'placekicker', 'pkyds',
       'pkresult', 'playyds', 'penalty', 'int', 'fumble', 'td', 'xp result',
       'two pt result', 'playstring', 'play str len', 'complex', 'max len',
       'max spaces', 'week'],
      dtype='object')

In [12]:
#remove 'o/u', 'home money', 'home spread', 'away money', 'away spread', 'playstring', 'play strn len', 'max len', 'max spaces'

drop_cols = ['o/u', 'home money', 'home spread', 'away money', 'away spread', 'playstring', 'play str len', 'max len', 'max spaces', 'provider']
pbp2019_red = pbp2019.drop(drop_cols, 1)
pbp2019_red.head()

,gameid,hometeam,awayteam,playid,activeid,activeteam,nonactive team,drive #,playtype,quarter,...,pkresult,playyds,penalty,int,fumble,td,xp result,two pt result,complex,week
0,401112431,Florida State Seminoles,Boise State Broncos,102925701,52,Florida State Seminoles,Boise State Broncos,5,7,2,...,NaN,16,False,False,True,False,NaN,NaN,5,1
1,401110725,LSU Tigers,Georgia Southern Eagles,101998701,290,Georgia Southern Eagles,LSU Tigers,3,7,1,...,NaN,10,False,False,True,False,NaN,NaN,5,1
2,401110723,Florida Gators,Miami Hurricanes,104966901,2390,Miami Hurricanes,Florida Gators,11,9,4,...,NaN,4,False,False,True,False,NaN,NaN,5,1
3,401121935,Troy Trojans,Campbell Fighting Camels,103979801,2097,Campbell Fighting Camels,Troy Trojans,9,20,3,...,NaN,0,False,False,True,False,NaN,NaN,5,1
4,401114228,Washington State Cougars,New Mexico State Aggies,101937404,265,Washington State Cougars,New Mexico State Aggies,2,24,1,...,NaN,0,False,False,False,False,NaN,NaN,2,1


In [13]:
#convert team names to lowercase

pbp2019_red['hometeam'] = pbp2019_red['hometeam'].str.lower()
pbp2019_red['awayteam'] = pbp2019_red['awayteam'].str.lower()
pbp2019_red['activeteam'] = pbp2019_red['activeteam'].str.lower()
pbp2019_red['nonactive team'] = pbp2019_red['nonactive team'].str.lower()

pbp2019_red.head()

,gameid,hometeam,awayteam,playid,activeid,activeteam,nonactive team,drive #,playtype,quarter,...,pkresult,playyds,penalty,int,fumble,td,xp result,two pt result,complex,week
0,401112431,florida state seminoles,boise state broncos,102925701,52,florida state seminoles,boise state broncos,5,7,2,...,NaN,16,False,False,True,False,NaN,NaN,5,1
1,401110725,lsu tigers,georgia southern eagles,101998701,290,georgia southern eagles,lsu tigers,3,7,1,...,NaN,10,False,False,True,False,NaN,NaN,5,1
2,401110723,florida gators,miami hurricanes,104966901,2390,miami hurricanes,florida gators,11,9,4,...,NaN,4,False,False,True,False,NaN,NaN,5,1
3,401121935,troy trojans,campbell fighting camels,103979801,2097,campbell fighting camels,troy trojans,9,20,3,...,NaN,0,False,False,True,False,NaN,NaN,5,1
4,401114228,washington state cougars,new mexico state aggies,101937404,265,washington state cougars,new mexico state aggies,2,24,1,...,NaN,0,False,False,False,False,NaN,NaN,2,1


In [16]:
# load name and mascot file

name_mascot = pd.read_csv('Other_Data/Names_Mascots.txt')
name_mascot.head()

,name,mascot
0,boston college,boston college eagles
1,clemson,clemson tigers
2,duke,duke blue devils
3,florida state,florida state seminoles
4,georgia tech,georgia tech yellow jackets


In [17]:
#convert to dict
mascot_dict = name_mascot.set_index('name')['mascot'].to_dict()
print(mascot_dict)

{'boston college': 'boston college eagles', 'clemson': 'clemson tigers', 'duke': 'duke blue devils', 'florida state': 'florida state seminoles', 'georgia tech': 'georgia tech yellow jackets', 'louisville': 'louisville cardinals', 'miami': 'miami hurricanes', 'north carolina': 'north carolina tar heels', 'nc state': 'nc state wolfpack', 'pittsburgh': 'pittsburgh panthers', 'syracuse': 'syracuse orange', 'virginia': 'virginia cavaliers', 'virginia tech': 'virginia tech hokies', 'wake forest': 'wake forest demon deacons', 'illinois': 'illinois fighting illini', 'indiana': 'indiana hoosiers', 'iowa': 'iowa hawkeyes', 'maryland': 'maryland terrapins', 'michigan': 'michigan wolverines', 'michigan state': 'michigan state spartans', 'minnesota': 'minnesota golden gophers', 'nebraska': 'nebraska cornhuskers', 'northwestern': 'northwestern wildcats', 'ohio state': 'ohio state buckeyes', 'penn state': 'penn state nittany lions', 'purdue': 'purdue boilermakers', 'rutgers': 'rutgers scarlet knights

In [18]:
#load conference data
conferences = pd.read_csv('Other_Data/Conferences.csv',skip_blank_lines=True).fillna(np.nan) 
conferences

#conver to lowercase
conferences = conferences.apply(lambda x: x.astype(str).str.lower())

In [19]:
#adding nan to dictionary
mascot_dict['nan']=np.nan
mascot_dict

{'boston college': 'boston college eagles',
 'clemson': 'clemson tigers',
 'duke': 'duke blue devils',
 'florida state': 'florida state seminoles',
 'georgia tech': 'georgia tech yellow jackets',
 'louisville': 'louisville cardinals',
 'miami': 'miami hurricanes',
 'north carolina': 'north carolina tar heels',
 'nc state': 'nc state wolfpack',
 'pittsburgh': 'pittsburgh panthers',
 'syracuse': 'syracuse orange',
 'virginia': 'virginia cavaliers',
 'virginia tech': 'virginia tech hokies',
 'wake forest': 'wake forest demon deacons',
 'illinois': 'illinois fighting illini',
 'indiana': 'indiana hoosiers',
 'iowa': 'iowa hawkeyes',
 'maryland': 'maryland terrapins',
 'michigan': 'michigan wolverines',
 'michigan state': 'michigan state spartans',
 'minnesota': 'minnesota golden gophers',
 'nebraska': 'nebraska cornhuskers',
 'northwestern': 'northwestern wildcats',
 'ohio state': 'ohio state buckeyes',
 'penn state': 'penn state nittany lions',
 'purdue': 'purdue boilermakers',
 'rutgers'

In [21]:
#changing team name in conferences to include mascot
for conf in conferences:
    conferences[conf] = conferences[conf].apply(lambda x: mascot_dict[x])

In [22]:
conferences

,ACC,Big Ten,Big 12,Pac-12,SEC
0,boston college eagles,illinois fighting illini,baylor bears,arizona wildcats,alabama crimson tide
1,clemson tigers,indiana hoosiers,iowa state cyclones,arizona state sun devils,arkansas razorbacks
2,duke blue devils,iowa hawkeyes,kansas jayhawks,california golden bears,auburn tigers
3,florida state seminoles,maryland terrapins,kansas state wildcats,ucla bruins,florida gators
4,georgia tech yellow jackets,michigan wolverines,oklahoma sooners,colorado buffaloes,georgia bulldogs
5,louisville cardinals,michigan state spartans,oklahoma state cowboys,oregon ducks,kentucky wildcats
6,miami hurricanes,minnesota golden gophers,tcu horned frogs,oregon state beavers,lsu tigers
7,north carolina tar heels,nebraska cornhuskers,texas longhorns,usc trojans,ole miss rebels
8,nc state wolfpack,northwestern wildcats,texas tech red raiders,stanford cardinal,mississippi state bulldogs
9,pittsburgh panthers,ohio state buckeyes,west virginia mountaineers,utah utes,missouri tigers


In [23]:
#convert conferences to dictionary
conf_dict = conferences.to_dict(orient='list')
conf_dict.keys()

dict_keys(['ACC', 'Big Ten', 'Big 12', 'Pac-12', 'SEC'])

In [24]:
# reverse dconferences dictionary
conf_rev = {v: k for k, values in conf_dict.items() for v in values}
print(conf_rev)

{'boston college eagles': 'ACC', 'clemson tigers': 'ACC', 'duke blue devils': 'ACC', 'florida state seminoles': 'ACC', 'georgia tech yellow jackets': 'ACC', 'louisville cardinals': 'ACC', 'miami hurricanes': 'ACC', 'north carolina tar heels': 'ACC', 'nc state wolfpack': 'ACC', 'pittsburgh panthers': 'ACC', 'syracuse orange': 'ACC', 'virginia cavaliers': 'ACC', 'virginia tech hokies': 'ACC', 'wake forest demon deacons': 'ACC', 'illinois fighting illini': 'Big Ten', 'indiana hoosiers': 'Big Ten', 'iowa hawkeyes': 'Big Ten', 'maryland terrapins': 'Big Ten', 'michigan wolverines': 'Big Ten', 'michigan state spartans': 'Big Ten', 'minnesota golden gophers': 'Big Ten', 'nebraska cornhuskers': 'Big Ten', 'northwestern wildcats': 'Big Ten', 'ohio state buckeyes': 'Big Ten', 'penn state nittany lions': 'Big Ten', 'purdue boilermakers': 'Big Ten', 'rutgers scarlet knights': 'Big Ten', 'wisconsin badgers': 'Big Ten', 'baylor bears': 'Big 12', 'iowa state cyclones': 'Big 12', 'kansas jayhawks': 'B

In [45]:
#lowercase 
keys_values = conf_rev.items()
conf_rev = {str(key).lower(): str(value).lower() for key, value in keys_values}
print(conf_rev)

{'boston college eagles': 'acc', 'clemson tigers': 'acc', 'duke blue devils': 'acc', 'florida state seminoles': 'acc', 'georgia tech yellow jackets': 'acc', 'louisville cardinals': 'acc', 'miami hurricanes': 'acc', 'north carolina tar heels': 'acc', 'nc state wolfpack': 'acc', 'pittsburgh panthers': 'acc', 'syracuse orange': 'acc', 'virginia cavaliers': 'acc', 'virginia tech hokies': 'acc', 'wake forest demon deacons': 'acc', 'illinois fighting illini': 'big ten', 'indiana hoosiers': 'big ten', 'iowa hawkeyes': 'big ten', 'maryland terrapins': 'big ten', 'michigan wolverines': 'big ten', 'michigan state spartans': 'big ten', 'minnesota golden gophers': 'big ten', 'nebraska cornhuskers': 'big ten', 'northwestern wildcats': 'big ten', 'ohio state buckeyes': 'big ten', 'penn state nittany lions': 'big ten', 'purdue boilermakers': 'big ten', 'rutgers scarlet knights': 'big ten', 'wisconsin badgers': 'big ten', 'baylor bears': 'big 12', 'iowa state cyclones': 'big 12', 'kansas jayhawks': 'b

In [27]:
#subset of pbp to test
pbp_small = pbp2019_red.iloc[:50,0:50]
pbp_small.head()

,gameid,hometeam,awayteam,playid,activeid,activeteam,nonactive team,drive #,playtype,quarter,...,pkresult,playyds,penalty,int,fumble,td,xp result,two pt result,complex,week
0,401112431,florida state seminoles,boise state broncos,102925701,52,florida state seminoles,boise state broncos,5,7,2,...,NaN,16,False,False,True,False,NaN,NaN,5,1
1,401110725,lsu tigers,georgia southern eagles,101998701,290,georgia southern eagles,lsu tigers,3,7,1,...,NaN,10,False,False,True,False,NaN,NaN,5,1
2,401110723,florida gators,miami hurricanes,104966901,2390,miami hurricanes,florida gators,11,9,4,...,NaN,4,False,False,True,False,NaN,NaN,5,1
3,401121935,troy trojans,campbell fighting camels,103979801,2097,campbell fighting camels,troy trojans,9,20,3,...,NaN,0,False,False,True,False,NaN,NaN,5,1
4,401114228,washington state cougars,new mexico state aggies,101937404,265,washington state cougars,new mexico state aggies,2,24,1,...,NaN,0,False,False,False,False,NaN,NaN,2,1


In [46]:
#add home conference column to pbp_small as test
pbp_small['home_conf']=np.nan
for k, v in conf_rev.items():
    for i in range(len(pbp_small['hometeam'])):
        if k in pbp_small['hometeam'][i]:
            pbp_small['home_conf'][i]=v


/Users/regan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/regan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [47]:
#add away conference column to pbp_small as test
pbp_small['away_conf']=np.nan
for k, v in conf_rev.items():
    for i in range(len(pbp_small['awayteam'])):
        if k in pbp_small['awayteam'][i]:
            pbp_small['away_conf'][i] = v

/Users/regan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [48]:
pbp_small

,gameid,hometeam,awayteam,playid,activeid,activeteam,nonactive team,drive #,playtype,quarter,...,penalty,int,fumble,td,xp result,two pt result,complex,week,home_conf,away_conf
0,401112431,florida state seminoles,boise state broncos,102925701,52,florida state seminoles,boise state broncos,5,7,2,...,False,False,True,False,NaN,NaN,5,1,acc,NaN
1,401110725,lsu tigers,georgia southern eagles,101998701,290,georgia southern eagles,lsu tigers,3,7,1,...,False,False,True,False,NaN,NaN,5,1,sec,NaN
2,401110723,florida gators,miami hurricanes,104966901,2390,miami hurricanes,florida gators,11,9,4,...,False,False,True,False,NaN,NaN,5,1,sec,acc
3,401121935,troy trojans,campbell fighting camels,103979801,2097,campbell fighting camels,troy trojans,9,20,3,...,False,False,True,False,NaN,NaN,5,1,NaN,NaN
4,401114228,washington state cougars,new mexico state aggies,101937404,265,washington state cougars,new mexico state aggies,2,24,1,...,False,False,False,False,NaN,NaN,2,1,pac-12,NaN
5,401114228,washington state cougars,new mexico state aggies,101937405,265,washington state cougars,new mexico state aggies,2,24,1,...,False,False,False,False,NaN,NaN,2,1,pac-12,NaN
6,401114228,washington state cougars,new mexico state aggies,102849902,265,washington state cougars,new mexico state aggies,3,24,2,...,False,False,False,False,NaN,NaN,2,1,pac-12,NaN
7,401114228,washington state cougars,new mexico state aggies,102935504,265,washington state cougars,new mexico state aggies,5,24,2,...,False,False,False,False,NaN,NaN,2,1,pac-12,NaN
8,401112212,michigan state spartans,tulsa golden hurricane,102955001,202,tulsa golden hurricane,michigan state spartans,6,7,2,...,False,False,True,False,NaN,NaN,5,1,big ten,NaN
9,401112238,nebraska cornhuskers,south alabama jaguars,101974201,6,south alabama jaguars,nebraska cornhuskers,3,7,1,...,False,False,True,False,NaN,NaN,5,1,big ten,NaN


In [49]:
#add home conference column
pbp2019_red['home_conf']=np.nan
for k, v in conf_rev.items():
    for i in range(len(pbp2019_red['hometeam'])):
        if k in pbp2019_red['hometeam'][i]:
            pbp2019_red['home_conf'][i]=v

/Users/regan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/regan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [50]:
#add away conference column
pbp2019_red['away_conf']=np.nan
for k, v in conf_rev.items():
    for i in range(len(pbp2019_red['awayteam'])):
        if k in pbp2019_red['awayteam'][i]:
            pbp2019_red['away_conf'][i] = v

/Users/regan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [51]:
pbp2019_red.head()

,gameid,hometeam,awayteam,playid,activeid,activeteam,nonactive team,drive #,playtype,quarter,...,penalty,int,fumble,td,xp result,two pt result,complex,week,home_conf,away_conf
0,401112431,florida state seminoles,boise state broncos,102925701,52,florida state seminoles,boise state broncos,5,7,2,...,False,False,True,False,NaN,NaN,5,1,acc,NaN
1,401110725,lsu tigers,georgia southern eagles,101998701,290,georgia southern eagles,lsu tigers,3,7,1,...,False,False,True,False,NaN,NaN,5,1,sec,NaN
2,401110723,florida gators,miami hurricanes,104966901,2390,miami hurricanes,florida gators,11,9,4,...,False,False,True,False,NaN,NaN,5,1,sec,acc
3,401121935,troy trojans,campbell fighting camels,103979801,2097,campbell fighting camels,troy trojans,9,20,3,...,False,False,True,False,NaN,NaN,5,1,NaN,NaN
4,401114228,washington state cougars,new mexico state aggies,101937404,265,washington state cougars,new mexico state aggies,2,24,1,...,False,False,False,False,NaN,NaN,2,1,pac-12,NaN


In [67]:
#drop all games that do not involve a team in the Big Ten or SEC
c = ['big ten','sec']
pbph = pbp2019_red.loc[pbp2019_red['home_conf'].isin(c)]
pbpa = pbp2019_red.loc[pbp2019_red['away_conf'].isin(c)]
pbp= pbph.append(pbpa)
#this will append many duplicates where the home and away team are both in the big ten and sec

In [68]:
# drop duplicates
pbp = pbp.drop_duplicates()

In [69]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [70]:
#check dataframe sizes
pbp2019_red.shape
pbp.shape

(134729, 41)

(34166, 41)

In [71]:
pbp.head()

,gameid,hometeam,awayteam,playid,activeid,activeteam,nonactive team,drive #,playtype,quarter,...,penalty,int,fumble,td,xp result,two pt result,complex,week,home_conf,away_conf
1,401110725,lsu tigers,georgia southern eagles,101998701,290,georgia southern eagles,lsu tigers,3,7,1,...,False,False,True,False,NaN,NaN,5,1,sec,NaN
2,401110723,florida gators,miami hurricanes,104966901,2390,miami hurricanes,florida gators,11,9,4,...,False,False,True,False,NaN,NaN,5,1,sec,acc
8,401112212,michigan state spartans,tulsa golden hurricane,102955001,202,tulsa golden hurricane,michigan state spartans,6,7,2,...,False,False,True,False,NaN,NaN,5,1,big ten,NaN
9,401112238,nebraska cornhuskers,south alabama jaguars,101974201,6,south alabama jaguars,nebraska cornhuskers,3,7,1,...,False,False,True,False,NaN,NaN,5,1,big ten,NaN
86,401110730,tennessee volunteers,georgia state panthers,104857801,2633,tennessee volunteers,georgia state panthers,8,24,4,...,False,False,False,False,NaN,NaN,2,1,sec,NaN


In [92]:
pbp['week'].max()

14

In [355]:
pbp.dtypes

gameid              int64
hometeam           object
awayteam           object
playid              int64
activeid            int64
activeteam         object
nonactive team     object
drive #             int64
playtype            int64
quarter             int64
sec                 int64
homescore           int64
awayscore           int64
down                int64
distance            int64
yardstoendzone      int64
zone               object
qb                 object
pass yds          float64
receiver           object
recyds            float64
runner             object
runyds            float64
kicker             object
kickyds           float64
kickreturner       object
kickretyds        float64
placekicker        object
pkyds             float64
pkresult           object
playyds             int64
penalty              bool
int                  bool
fumble               bool
td                   bool
xp result          object
two pt result      object
complex             int64
week        

In [378]:
#load rankings dataset
rankings2019 = pd.read_csv('Other_Data/Ranking.csv')
rankings2019

,PreseasonAug 1[18],Week 1Sep 3[19],Week 2Sep 8[20],Week 3Sep 15[21],Week 4Sep 22[22],Week 5Sep 29[23],Week 6Oct 6[24],Week 7Oct 13[25],Week 8Oct 20[26],Week 9Oct 27[27],Week 10Nov 3[28],Week 11Nov 10[29],Week 12Nov 17[30],Week 13Nov 24[31],Week 14Dec 1[32],Week 15Dec 8[33],Week 16 (Final)Jan 14[34],Unnamed: 17
0,Clemson (59),Clemson (1–0) (58),Clemson (2–0) (60),Clemson (3–0) (62),Clemson (4–0) (62),Alabama (5–0) (29),Alabama (5–0) (42),Alabama (6–0) (44),Alabama (7–0) (44),Alabama (8–0) (40),Alabama (8–0) (37),LSU (9–0) (55),LSU (10–0) (55),LSU (11–0) (52),LSU (12–0) (43),LSU (13–0) (46),LSU (15–0) (65),1
1,Alabama (6),Alabama (1–0) (6),Alabama (2–0) (3),Alabama (3–0) (3),Alabama (4–0) (2),Clemson (5–0) (30),Clemson (5–0) (20),Clemson (6–0) (14),Clemson (7–0) (10),LSU (8–0) (7),LSU (8–0) (11),Ohio State (9–0) (5),Ohio State (10–0) (6),Ohio State (11–0) (7),Ohio State (12–0) (17),Ohio State (13–0) (14),Clemson (14–1),2
2,Georgia,Georgia (1–0),Georgia (2–0),Georgia (3–0),Georgia (4–0) (1),Georgia (4–0) (1),Georgia (5–0),LSU (6–0) (3),LSU (7–0) (3),Clemson (8–0) (10),Clemson (9–0) (9),Clemson (10–0) (4),Clemson (11–0) (4),Clemson (11–0) (4),Clemson (12–0) (4),Clemson (13–0) (5),Ohio State (13–1),3
3,Oklahoma,Oklahoma (1–0),Oklahoma (2–0),Oklahoma (3–0),Oklahoma (3–0),Oklahoma (4–0),Ohio State (5–0) (3),Ohio State (6–0) (4),Ohio State (7–0) (8),Ohio State (8–0) (8),Ohio State (8–0) (8),Alabama (8–1),Georgia (9–1),Georgia (10–1),Georgia (11–1),Oklahoma (12–1),Georgia (12–2),4
4,Ohio State,Ohio State (1–0),LSU (2–0),LSU (3–0),LSU (4–0),Ohio State (5–0) (4),Oklahoma (5–0),Oklahoma (6–0),Oklahoma (7–0),Penn State (8–0),Penn State (8–0),Georgia (8–1),Alabama (9–1),Alabama (10–1),Utah (11–1),Georgia (11–2),Oregon (12–2),5
5,LSU,LSU (1–0),Ohio State (2–0),Ohio State (3–0),Ohio State (4–0),LSU (4–0),LSU (5–0),Wisconsin (6–0),Penn State (7–0),Florida (7–1),Georgia (7–1),Oregon (8–1),Oregon (9–1),Utah (10–1),Oklahoma (11–1),Oregon (11–2),Oklahoma (12–2),6
6,Michigan,Michigan (1–0),Notre Dame (2–0),Notre Dame (2–0),Auburn (4–0),Auburn (4–0),Florida (6–0),Penn State (6–0),Notre Dame (5–1),Georgia (6–1),Oregon (8–1),Minnesota (9–0),Oklahoma (9–1),Oklahoma (10–1),Florida (10–2),Florida (10–2),Florida (11–2),7
7,Florida,Notre Dame (1–0),Florida (2–0),Florida (3–0),Florida (4–0),Florida (5–0),Wisconsin (5–0),Notre Dame (5–1),Florida (7–1),Oregon (7–1),Oklahoma (7–1),Oklahoma (8–1),Utah (9–1),Florida (9–2),Baylor (11–1),Baylor (11–2),Alabama (11–2),8
8,Notre Dame,Texas (1–0),Auburn (2–0),Auburn (3–0),Wisconsin (3–0),Wisconsin (4–0),Penn State (5–0),Florida (6–1),Georgia (6–1),Oklahoma (7–1),Utah (8–1),Utah (8–1),Penn State (9–1),Minnesota (10–1),Alabama (10–2),Alabama (10–2),Penn State (11–2),9
9,Texas,Florida (1–0),Michigan (2–0),Michigan (2–0),Notre Dame (2–1),Notre Dame (3–1),Notre Dame (4–1),Georgia (5–1),Auburn (6–1),Utah (7–1),Baylor (8–0),Baylor (9–0),Florida (9–2),Baylor (10–1),Wisconsin (10–2),Utah (11–2),Minnesota (11–2),10


In [379]:
rankings2019col = rankings2019.columns

#remove []
rankings2019col_rename = [re.sub('\[.*?\]', '', name) for name in rankings2019col]

In [380]:
#remove date
rankings2019col_rename2 = [re.sub('Aug [0-9]*| Jan [0-9]*|Sep [0-9]*|Oct [0-9]*|Nov [0-9]*|Dec [0-9]*|Jan [0-9]*','',month) for month in rankings2019col_rename]
print(rankings2019col_rename2)

['Preseason', 'Week 1', 'Week 2', 'Week 3', 'Week 4', 'Week 5', 'Week 6', 'Week 7', 'Week 8', 'Week 9', 'Week 10', 'Week 11', 'Week 12', 'Week 13', 'Week 14', 'Week 15', 'Week 16 (Final)', 'Unnamed: 17']


In [381]:
#remove "week"
rankings2019col_3 = [re.sub('Week','',week)for week in rankings2019col_rename2]

In [382]:
#replace columns with new names
rankings2019.columns = rankings2019col_3
rankings2019.head()

,Preseason,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16 (Final),Unnamed: 17
0,Clemson (59),Clemson (1–0) (58),Clemson (2–0) (60),Clemson (3–0) (62),Clemson (4–0) (62),Alabama (5–0) (29),Alabama (5–0) (42),Alabama (6–0) (44),Alabama (7–0) (44),Alabama (8–0) (40),Alabama (8–0) (37),LSU (9–0) (55),LSU (10–0) (55),LSU (11–0) (52),LSU (12–0) (43),LSU (13–0) (46),LSU (15–0) (65),1
1,Alabama (6),Alabama (1–0) (6),Alabama (2–0) (3),Alabama (3–0) (3),Alabama (4–0) (2),Clemson (5–0) (30),Clemson (5–0) (20),Clemson (6–0) (14),Clemson (7–0) (10),LSU (8–0) (7),LSU (8–0) (11),Ohio State (9–0) (5),Ohio State (10–0) (6),Ohio State (11–0) (7),Ohio State (12–0) (17),Ohio State (13–0) (14),Clemson (14–1),2
2,Georgia,Georgia (1–0),Georgia (2–0),Georgia (3–0),Georgia (4–0) (1),Georgia (4–0) (1),Georgia (5–0),LSU (6–0) (3),LSU (7–0) (3),Clemson (8–0) (10),Clemson (9–0) (9),Clemson (10–0) (4),Clemson (11–0) (4),Clemson (11–0) (4),Clemson (12–0) (4),Clemson (13–0) (5),Ohio State (13–1),3
3,Oklahoma,Oklahoma (1–0),Oklahoma (2–0),Oklahoma (3–0),Oklahoma (3–0),Oklahoma (4–0),Ohio State (5–0) (3),Ohio State (6–0) (4),Ohio State (7–0) (8),Ohio State (8–0) (8),Ohio State (8–0) (8),Alabama (8–1),Georgia (9–1),Georgia (10–1),Georgia (11–1),Oklahoma (12–1),Georgia (12–2),4
4,Ohio State,Ohio State (1–0),LSU (2–0),LSU (3–0),LSU (4–0),Ohio State (5–0) (4),Oklahoma (5–0),Oklahoma (6–0),Oklahoma (7–0),Penn State (8–0),Penn State (8–0),Georgia (8–1),Alabama (9–1),Alabama (10–1),Utah (11–1),Georgia (11–2),Oregon (12–2),5


In [383]:
#remove (record) and (votes) from data
rankings = rankings2019.replace(to_replace = '\(.*?\)', value = '', regex=True)
rankings.head()

,Preseason,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16 (Final),Unnamed: 17
0,Clemson,Clemson,Clemson,Clemson,Clemson,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,LSU,LSU,LSU,LSU,LSU,LSU,1
1,Alabama,Alabama,Alabama,Alabama,Alabama,Clemson,Clemson,Clemson,Clemson,LSU,LSU,Ohio State,Ohio State,Ohio State,Ohio State,Ohio State,Clemson,2
2,Georgia,Georgia,Georgia,Georgia,Georgia,Georgia,Georgia,LSU,LSU,Clemson,Clemson,Clemson,Clemson,Clemson,Clemson,Clemson,Ohio State,3
3,Oklahoma,Oklahoma,Oklahoma,Oklahoma,Oklahoma,Oklahoma,Ohio State,Ohio State,Ohio State,Ohio State,Ohio State,Alabama,Georgia,Georgia,Georgia,Oklahoma,Georgia,4
4,Ohio State,Ohio State,LSU,LSU,LSU,Ohio State,Oklahoma,Oklahoma,Oklahoma,Penn State,Penn State,Georgia,Alabama,Alabama,Utah,Georgia,Oregon,5


In [384]:
rankings.columns

Index(['Preseason', ' 1', ' 2', ' 3', ' 4', ' 5', ' 6', ' 7', ' 8', ' 9',
       ' 10', ' 11', ' 12', ' 13', ' 14', ' 15', ' 16 (Final)', 'Unnamed: 17'],
      dtype='object')

In [385]:
#drop preseason and postseason rankings
drop_col_rank = ['Preseason',' 15',' 16 (Final)','Unnamed: 17']
rank = rankings.drop(drop_col_rank, 1)
rank.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Clemson,Clemson,Clemson,Clemson,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,LSU,LSU,LSU,LSU
1,Alabama,Alabama,Alabama,Alabama,Clemson,Clemson,Clemson,Clemson,LSU,LSU,Ohio State,Ohio State,Ohio State,Ohio State
2,Georgia,Georgia,Georgia,Georgia,Georgia,Georgia,LSU,LSU,Clemson,Clemson,Clemson,Clemson,Clemson,Clemson
3,Oklahoma,Oklahoma,Oklahoma,Oklahoma,Oklahoma,Ohio State,Ohio State,Ohio State,Ohio State,Ohio State,Alabama,Georgia,Georgia,Georgia
4,Ohio State,LSU,LSU,LSU,Ohio State,Oklahoma,Oklahoma,Oklahoma,Penn State,Penn State,Georgia,Alabama,Alabama,Utah


In [389]:
#convert column names to integers
colnames = [1,2,3,4,5,6,7,8,9,10,11,12,13, 14]
col = rank.columns
rank = rank.rename(columns = dict(zip(col, colnames)))
rank.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Clemson,Clemson,Clemson,Clemson,Alabama,Alabama,Alabama,Alabama,Alabama,Alabama,LSU,LSU,LSU,LSU
1,Alabama,Alabama,Alabama,Alabama,Clemson,Clemson,Clemson,Clemson,LSU,LSU,Ohio State,Ohio State,Ohio State,Ohio State
2,Georgia,Georgia,Georgia,Georgia,Georgia,Georgia,LSU,LSU,Clemson,Clemson,Clemson,Clemson,Clemson,Clemson
3,Oklahoma,Oklahoma,Oklahoma,Oklahoma,Oklahoma,Ohio State,Ohio State,Ohio State,Ohio State,Ohio State,Alabama,Georgia,Georgia,Georgia
4,Ohio State,LSU,LSU,LSU,Ohio State,Oklahoma,Oklahoma,Oklahoma,Penn State,Penn State,Georgia,Alabama,Alabama,Utah


In [390]:
#check that column names are integers
rank.columns

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64')

In [391]:
#lowercase 
rank = rank.apply(lambda x: x.astype(str).str.lower())
rank.head(2)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,clemson,clemson,clemson,clemson,alabama,alabama,alabama,alabama,alabama,alabama,lsu,lsu,lsu,lsu
1,alabama,alabama,alabama,alabama,clemson,clemson,clemson,clemson,lsu,lsu,ohio state,ohio state,ohio state,ohio state


In [392]:
#rename schools in ranking to match pbp
for school, mascot in mascot_dict.items():
    for team in rank:
        for i in range(len(rank)):
            if school in rank[team][i]:
                rank[team][i] = mascot

In [393]:
rank.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,clemson tigers,clemson tigers,clemson tigers,clemson tigers,alabama crimson tide,alabama crimson tide,alabama crimson tide,alabama crimson tide,alabama crimson tide,alabama crimson tide,lsu tigers,lsu tigers,lsu tigers,lsu tigers
1,alabama crimson tide,alabama crimson tide,alabama crimson tide,alabama crimson tide,clemson tigers,clemson tigers,clemson tigers,clemson tigers,lsu tigers,lsu tigers,ohio state buckeyes,ohio state buckeyes,ohio state buckeyes,ohio state buckeyes
2,georgia bulldogs,georgia bulldogs,georgia bulldogs,georgia bulldogs,georgia bulldogs,georgia bulldogs,lsu tigers,lsu tigers,clemson tigers,clemson tigers,clemson tigers,clemson tigers,clemson tigers,clemson tigers
3,oklahoma sooners,oklahoma sooners,oklahoma sooners,oklahoma sooners,oklahoma sooners,ohio state buckeyes,ohio state buckeyes,ohio state buckeyes,ohio state buckeyes,ohio state buckeyes,alabama crimson tide,georgia bulldogs,georgia bulldogs,georgia bulldogs
4,ohio state buckeyes,lsu tigers,lsu tigers,lsu tigers,ohio state buckeyes,oklahoma sooners,oklahoma sooners,oklahoma sooners,penn state nittany lions,penn state nittany lions,georgia bulldogs,alabama crimson tide,alabama crimson tide,utah utes


In [395]:
rank_25 = rank

In [396]:
#reset index to correspond to ranking
rank_25.index = np.arange(1, len(rank_25) +1)

In [397]:
#convert rank_25 to dict
rank_dict = rank_25.to_dict()
rank_dict

{1: {1: 'clemson tigers',
  2: 'alabama crimson tide',
  3: 'georgia bulldogs',
  4: 'oklahoma sooners',
  5: 'ohio state buckeyes',
  6: 'lsu tigers',
  7: 'michigan wolverines',
  8: 'notre dame ',
  9: 'texas longhorns',
  10: 'florida gators',
  11: 'texas longhorns',
  12: 'washington huskies',
  13: 'auburn tigers',
  14: 'penn state nittany lions',
  15: 'utah utes',
  16: 'wisconsin badgers',
  17: 'ucf ',
  18: 'oregon ducks',
  19: 'iowa hawkeyes',
  20: 'michigan wolverines',
  21: 'washington huskies',
  22: 'syracuse orange',
  23: 'stanford cardinal',
  24: 'boise state ',
  25: 'nebraska cornhuskers'},
 2: {1: 'clemson tigers',
  2: 'alabama crimson tide',
  3: 'georgia bulldogs',
  4: 'oklahoma sooners',
  5: 'lsu tigers',
  6: 'ohio state buckeyes',
  7: 'notre dame ',
  8: 'florida gators',
  9: 'auburn tigers',
  10: 'michigan wolverines',
  11: 'penn state nittany lions',
  12: 'utah utes',
  13: 'texas longhorns',
  14: 'wisconsin badgers',
  15: 'texas longhorns',

In [398]:
#reverse inner dictionary
from collections import defaultdict
import pprint

flipped = defaultdict(dict)
for week, spots in rank_dict.items():
    for spot, team in spots.items():
        flipped[week][team]= spot

pprint.pprint(dict(flipped))

{1: {'alabama crimson tide': 2,
     'auburn tigers': 13,
     'boise state ': 24,
     'clemson tigers': 1,
     'florida gators': 10,
     'georgia bulldogs': 3,
     'iowa hawkeyes': 19,
     'lsu tigers': 6,
     'michigan wolverines': 20,
     'nebraska cornhuskers': 25,
     'notre dame ': 8,
     'ohio state buckeyes': 5,
     'oklahoma sooners': 4,
     'oregon ducks': 18,
     'penn state nittany lions': 14,
     'stanford cardinal': 23,
     'syracuse orange': 22,
     'texas longhorns': 11,
     'ucf ': 17,
     'utah utes': 15,
     'washington huskies': 21,
     'wisconsin badgers': 16},
 2: {'alabama crimson tide': 2,
     'auburn tigers': 9,
     'boise state ': 22,
     'clemson tigers': 1,
     'florida gators': 8,
     'georgia bulldogs': 3,
     'iowa hawkeyes': 18,
     'lsu tigers': 5,
     'maryland terrapins': 25,
     'michigan wolverines': 19,
     'mississippi state bulldogs': 23,
     'notre dame ': 7,
     'ohio state buckeyes': 6,
     'oklahoma sooners': 4

In [427]:
#add home rank column
replaced_dfs = []
for week, pbp_df in pbp.groupby('week'):
    pbp_df['home_rank'] = pbp_df['hometeam'].map(flipped[week])
    replaced_dfs.append(pbp_df)

In [428]:
pbp_rank = pd.concat(replaced_dfs)
pbp_rank.head()

,gameid,hometeam,awayteam,playid,activeid,activeteam,nonactive team,drive #,playtype,quarter,...,int,fumble,td,xp result,two pt result,complex,week,home_conf,away_conf,home_rank
1,401110725,lsu tigers,georgia southern eagles,101998701,290,georgia southern eagles,lsu tigers,3,7,1,...,False,True,False,NaN,NaN,5,1,sec,NaN,6.0
2,401110723,florida gators,miami hurricanes,104966901,2390,miami hurricanes,florida gators,11,9,4,...,False,True,False,NaN,NaN,5,1,sec,acc,10.0
8,401112212,michigan state spartans,tulsa golden hurricane,102955001,202,tulsa golden hurricane,michigan state spartans,6,7,2,...,False,True,False,NaN,NaN,5,1,big ten,NaN,NaN
9,401112238,nebraska cornhuskers,south alabama jaguars,101974201,6,south alabama jaguars,nebraska cornhuskers,3,7,1,...,False,True,False,NaN,NaN,5,1,big ten,NaN,25.0
86,401110730,tennessee volunteers,georgia state panthers,104857801,2633,tennessee volunteers,georgia state panthers,8,24,4,...,False,False,False,NaN,NaN,2,1,sec,NaN,NaN


In [429]:
#check dataframe size
pbp_rank.shape

(34166, 42)

In [ ]:
pbp_rank['away_rank']=np.nan

In [431]:
#add away rank column
replaced_away_dfs = []
for week, pbp_df in pbp_rank.groupby('week'):
    pbp_df['away_rank'] = pbp_df['awayteam'].map(flipped[week])
    replaced_away_dfs.append(pbp_df)

In [436]:
pbp_rank = pd.concat(replaced_away_dfs)
pbp_rank.head(25)

,gameid,hometeam,awayteam,playid,activeid,activeteam,nonactive team,drive #,playtype,quarter,...,fumble,td,xp result,two pt result,complex,week,home_conf,away_conf,home_rank,away_rank
1,401110725,lsu tigers,georgia southern eagles,101998701,290,georgia southern eagles,lsu tigers,3,7,1,...,True,False,NaN,NaN,5,1,sec,NaN,6.0,NaN
2,401110723,florida gators,miami hurricanes,104966901,2390,miami hurricanes,florida gators,11,9,4,...,True,False,NaN,NaN,5,1,sec,acc,10.0,NaN
8,401112212,michigan state spartans,tulsa golden hurricane,102955001,202,tulsa golden hurricane,michigan state spartans,6,7,2,...,True,False,NaN,NaN,5,1,big ten,NaN,NaN,NaN
9,401112238,nebraska cornhuskers,south alabama jaguars,101974201,6,south alabama jaguars,nebraska cornhuskers,3,7,1,...,True,False,NaN,NaN,5,1,big ten,NaN,25.0,NaN
86,401110730,tennessee volunteers,georgia state panthers,104857801,2633,tennessee volunteers,georgia state panthers,8,24,4,...,False,False,NaN,NaN,2,1,sec,NaN,NaN,NaN
87,401110730,tennessee volunteers,georgia state panthers,104994406,2633,tennessee volunteers,georgia state panthers,12,24,4,...,False,False,NaN,NaN,2,1,sec,NaN,NaN,NaN
88,401110730,tennessee volunteers,georgia state panthers,101884406,2633,tennessee volunteers,georgia state panthers,2,24,1,...,False,False,NaN,NaN,2,1,sec,NaN,NaN,NaN
89,401110730,tennessee volunteers,georgia state panthers,101979201,2633,tennessee volunteers,georgia state panthers,3,3,1,...,False,False,NaN,NaN,2,1,sec,NaN,NaN,NaN
90,401110732,vanderbilt commodores,georgia bulldogs,102876901,238,vanderbilt commodores,georgia bulldogs,2,24,2,...,False,False,NaN,NaN,2,1,sec,sec,NaN,3.0
94,401110732,vanderbilt commodores,georgia bulldogs,102866901,238,vanderbilt commodores,georgia bulldogs,2,5,2,...,False,False,NaN,NaN,2,1,sec,sec,NaN,3.0


In [437]:
#check dataframe rank
pbp_rank.shape

(34166, 43)